In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pandas as pd

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1S4VZ2I2IiM4EfGO4sjIsbiR7HKXkfXPlZfPn-ZOkNQg"

In [2]:
MOVIES_LIST = [
    "Anatomia de uma queda", 
    "The dark knight", 
    "Retorno_do_Rei", 
    "Matrix",
    "the_untouchables",
    "taxi_driver",
    "reservoir_dogs",
    "driver",
    "alone",
    "dungeon_meshi",
    "lakewood"
]
df = pd.DataFrame()

In [3]:
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    
    for movie in MOVIES_LIST:
        result = (
            sheet.values()
            .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=f'{movie}!A1:C80')
            .execute()
        )
        values = result.get("values", [])

        if not values:
            print("No data found.")
            exit()

        effects_list = [[movie, value[0], value[1], value[2]] for value in values if value and value[2] != 'label']
        new_df = pd.DataFrame(effects_list, columns=['filme', 'tempo_inicio', 'tempo_fim', 'efeito'])
        df=pd.concat([df, new_df], ignore_index=True)

except HttpError as err:
    print(err)


In [4]:
df

,filme,tempo_inicio,tempo_fim,efeito
0,Anatomia de uma queda,00:01:04.583,00:01:08.541,bolaQuicando
1,Anatomia de uma queda,00:01:25.624,00:01:41.000,chuveiroLigado
2,Anatomia de uma queda,00:02:20.750,00:02:22.333,cachorroSacudindo
3,Anatomia de uma queda,00:02:30.166,00:05:53.583,musicaAlta
4,Anatomia de uma queda,00:05:58.083,00:06:09.259,passos
...,...,...,...,...
146,dungeon_meshi,00:00:22.230,00:00:22.731,cortaAlimento
147,dungeon_meshi,00:00:24.774,00:00:25.984,vapor
148,lakewood,00:00:02.016,00:00:03.250,batidaPorta
149,lakewood,00:00:19.400,00:00:22.866,passos


In [5]:
df['efeito'].value_counts()

efeito
passos                               18
tiro                                 13
flash                                 9
tiros                                 8
flash                                 8
motor                                 6
mexendoLouça                          5
rugido                                5
musicaAlta                            5
vidroQuebrando                        4
explosao                              4
flashes                               3
explosao                              3
pneuDerrapando                        3
portaFechando                         3
soco                                  2
motorCarroExterno                     2
tiroteio                              2
barulhoAlto                           2
liquidoSendoColocadoNumRecipiente     2
cortaAlimento                         2
trovão                                2
diversos                              2
carro                                 2
motorCarroDentro                 

In [6]:
df.groupby('filme')['efeito'].value_counts()

filme                  efeito        
Anatomia de uma queda  passos            11
                       flash              9
                       flash              7
                       mexendoLouça       5
                       musicaAlta         5
                                         ..
the_untouchables       explosao           1
                       flash              1
                       motor              1
                       tiro               1
                       vidroQuebrando     1
Name: count, Length: 80, dtype: int64